# KNN 

Bu çalışmada veri seti üzerinde k-en yakın komşu(knn) algoritması kullanacağız. Knn algoritmasının çalışma mantığı girilen gözlemin en yakın k komşusuna bakarak çıktısını tahmin etmeye çalışmasıdır.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn import model_selection
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
#Veri ön işleme aşamalarını yapalım.
data=pd.read_csv("Hitters.csv")
df=data.copy()
df=df.dropna()
dms=pd.get_dummies(df[["League","Division","NewLeague"]])
y=df["Salary"]
x=df.drop(["Salary","League","Division","NewLeague"],axis=1).astype("float64")
x=pd.concat([x,dms[["League_N","Division_W","NewLeague_N"]]],axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [3]:
#KNN algoritmasının hiper parametresi komşuluk(k) sayısıdır.Optimize etmek gerekir.
#n_neighbors parametresiyle ayarlanır.Ön tanımlı olarak 5 verilmiştir.
knn=KNeighborsRegressor().fit(x_train,y_train)

In [4]:
knn

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [5]:
y_pred=knn.predict(x_test)

In [6]:
"Modelin test hatası:",np.sqrt(mean_squared_error(y_test,y_pred))

('Modelin test hatası:', 426.6570764525201)

In [7]:
#k sayısını değiştirerek eğitim hatasının nasıl değiştiğine bakalım.
RMSE=[]

for k in range(10):
    k=k+1
    knn_model=KNeighborsRegressor(n_neighbors=k).fit(x_train,y_train)
    y_pred2=knn_model.predict(x_train)
    rmse=np.sqrt(mean_squared_error(y_train,y_pred2))
    RMSE.append(rmse)
    print("k= ",k,"RMSE= ",rmse)

k=  1 RMSE=  0.0
k=  2 RMSE=  179.52761335480352
k=  3 RMSE=  205.20157172291863
k=  4 RMSE=  220.5139794876305
k=  5 RMSE=  239.64671325413764
k=  6 RMSE=  243.5904190007242
k=  7 RMSE=  258.1478781634636
k=  8 RMSE=  266.05374203349805
k=  9 RMSE=  269.73782093553376
k=  10 RMSE=  271.2798300436963


In [8]:
#Modelin optimum parametresini bulmak için hazır bir fonksiyon kullanacağız.
#GridSearchCV fonksiyonu, olası parametre seti verilerek olası kombinasyonları dener ve optimum olanını verir.
from sklearn.model_selection import GridSearchCV

In [9]:
#Parametre seti oluştururken parametre isminin fonksiyonda olan ile birebir olmasına dikkat edilmelidir.
#GridSearchCV sözlük içinde parametreyi arar ve değerleri dener.
knn_parametre={"n_neighbors":np.arange(1,15,1)}

In [10]:
knn2=KNeighborsRegressor()

In [11]:
knn_cv=GridSearchCV(knn2,knn_parametre,cv=10)

In [12]:
knn_cv.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [13]:
#Fonksiyonun bulduğu optimum parametreyi görelim.
knn_cv.best_params_["n_neighbors"]

8

In [14]:
#Valide edilmeden önceki ve sonraki eğitim hatalarını karşılaştırıp hangisini kullanmamız gerektiğine bakalım.
RMSE=[]
RMSE_CV=[]
for k in range(10):
    k=k+1
    knn_model=KNeighborsRegressor(n_neighbors=k).fit(x_train,y_train)
    y_pred2=knn_model.predict(x_train)
    rmse=np.sqrt(mean_squared_error(y_train,y_pred2))
    rmse_cv=np.sqrt(-1*cross_val_score(knn_model,x_train,y_train,cv=10,scoring="neg_mean_squared_error").mean())
    RMSE.append(rmse)
    RMSE_CV.append(rmse_cv)
    print("k= ",k,"RMSE= ",rmse,"    RMSE_CV= ",rmse_cv)
#Cross validation işlemi yapmadan önceki hata değerler daha düşük fakat 179-271 aralığında standart sapması yüksek.
#Valide edilmiş hata değerleri 283-301 aralığında standart sapması daha düşük, daha güvenilir bir sonuçlar.
#Modelin eğitim için hata değerlendirmesini valide edilmiş model üzerinden değerlendirmek daha doğru olur.

k=  1 RMSE=  0.0     RMSE_CV=  325.39475147063825
k=  2 RMSE=  179.52761335480352     RMSE_CV=  293.24000183333817
k=  3 RMSE=  205.20157172291863     RMSE_CV=  283.7486667487823
k=  4 RMSE=  220.5139794876305     RMSE_CV=  286.3240222024089
k=  5 RMSE=  239.64671325413764     RMSE_CV=  290.0705466132226
k=  6 RMSE=  243.5904190007242     RMSE_CV=  298.1263115575851
k=  7 RMSE=  258.1478781634636     RMSE_CV=  294.77070479194987
k=  8 RMSE=  266.05374203349805     RMSE_CV=  291.98672028891235
k=  9 RMSE=  269.73782093553376     RMSE_CV=  295.7162739573105
k=  10 RMSE=  271.2798300436963     RMSE_CV=  301.31047022701154


In [15]:
#Optimum komşuluk sayısı ile final modelimizi kuralım.
knn_tuned=KNeighborsRegressor(n_neighbors=8).fit(x_train,y_train)

In [16]:
y_pred_tuned=knn_tuned.predict(x_test)

In [17]:
"Tune edilmiş modelin test hatası:",np.sqrt(mean_squared_error(y_test,y_pred_tuned))

('Tune edilmiş modelin test hatası:', 413.7094731463598)